# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [79]:
#import csv file from weather_py assignment
city_data = pd.read_csv('../output_data/cities.csv')
city_data.head()

,Unnamed: 0,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
0,0,Hithadhoo,-0.60,73.08,83.28,71,22,4.50,MV,1597546061
1,1,Pemangkat,1.17,108.97,79.72,78,86,6.71,ID,1597546061
2,2,North Bend,43.41,-124.22,73.00,63,1,5.82,US,1597546061
3,3,Porto Walter,-8.27,-72.74,76.71,75,6,2.55,BR,1597546061
4,4,Albany,42.60,-73.97,71.01,66,74,5.82,US,1597546061


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [80]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[['lat', 'lng']]
#set weight to humidity
weights = city_data['humidity']
# Create a humidity Heatmap layer with a zoom level and set near/on San Antonio
fig = gmaps.figure(zoom_level=2, center=(29.42412, -98.49363))
#add heat map layer with max intensity and 5 point radius
heat_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [81]:
#narrow down list of cities to temp that is lower than 75 but greater than 60, humidity at 40 or less, and clouds less than 75%
temp_cities = city_data[city_data.max_temp < 75]
print(temp_cities.shape)
low_humid = temp_cities[temp_cities.humidity < 40]
print(low_humid.shape)
clouds = low_humid[low_humid.cloudiness < 75]
print(clouds.shape)
vacay_prefs = clouds[clouds.max_temp > 60]
vacay_prefs

(357, 10)
(14, 10)
(10, 10)


,Unnamed: 0,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
155,155,Singleton,-32.57,151.18,64.99,38,17,18.99,AU,1597546090
194,194,Saint Paul Harbor,57.79,-152.41,73.40,33,1,12.75,US,1597546096
289,289,Khandyga,62.67,135.60,62.73,30,0,7.58,RU,1597546113
308,308,Gladstone,-23.85,151.25,73.40,31,0,2.24,AU,1597546116
312,312,Vilhena,-12.74,-60.15,74.84,38,66,4.23,BR,1597546118
391,391,Alamosa,37.47,-105.87,69.80,30,1,10.29,US,1597546131
482,482,Ekibastuz,51.67,75.37,70.09,37,0,12.82,KZ,1597546147
509,509,Port Macquarie,-31.43,152.92,71.01,32,0,7.00,AU,1597546152


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [82]:
hotel_df = vacay_prefs

# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

C:\Users\gamem\anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\gamem\anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Hotel not found... skipping.


In [83]:
#remove cities without hotel data
hotel_df = hotel_df.dropna()
hotel_df

,Unnamed: 0,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date,Hotel Name
155,155,Singleton,-32.57,151.18,64.99,38,17,18.99,AU,1597546090,Quest Singleton
194,194,Saint Paul Harbor,57.79,-152.41,73.40,33,1,12.75,US,1597546096,Best Western Kodiak Inn and Convention Center
289,289,Khandyga,62.67,135.60,62.73,30,0,7.58,RU,1597546113,Apel'sin
308,308,Gladstone,-23.85,151.25,73.40,31,0,2.24,AU,1597546116,Highpoint International
312,312,Vilhena,-12.74,-60.15,74.84,38,66,4.23,BR,1597546118,Hotel Mirage
391,391,Alamosa,37.47,-105.87,69.80,30,1,10.29,US,1597546131,Best Western Alamosa Inn
509,509,Port Macquarie,-31.43,152.92,71.01,32,0,7.00,AU,1597546152,Rydges Port Macquarie


In [84]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [87]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

#combine heat and marker maps
fig = gmaps.figure(zoom_level=2, center=(29.42412, -98.49363))
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))